In [51]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [52]:
#過去の試合一覧が乗っているHPを取得
url = "https://www.football-lab.jp/nago/match/?year=2019"
res = requests.get(url)

In [53]:
#beautifulsoup(解析対象のHTML/XML, 利用するパーサー)
soup = BeautifulSoup(res.text, "html.parser")


In [54]:
#home のみ取り出す
home_aways = soup.find_all('span', class_='home')
home_url=[]
for i in home_aways:
#    home_away = soup.find('span', class_='away') #awayを見つけ出す
    Parent = i.parent.parent #親をたどる
    elems = Parent.find('a').next_element.next_element.next_element.find('a')['href'] #子要素を取り出す。
    home_url.append(elems)
home_url

['/nago/report/?year=2019&month=03&date=02',
 '/nago/report/?year=2019&month=03&date=30',
 '/nago/report/?year=2019&month=04&date=20',
 '/nago/report/?year=2019&month=04&date=28',
 '/nago/report/?year=2019&month=05&date=12',
 '/nago/report/?year=2019&month=05&date=26',
 '/nago/report/?year=2019&month=06&date=22',
 '/nago/report/?year=2019&month=07&date=07',
 '/nago/report/?year=2019&month=07&date=20',
 '/nago/report/?year=2019&month=08&date=10',
 '/nago/report/?year=2019&month=08&date=24',
 '/nago/report/?year=2019&month=08&date=30',
 '/nago/report/?year=2019&month=10&date=05',
 '/nago/report/?year=2019&month=10&date=19',
 '/nago/report/?year=2019&month=11&date=09',
 '/nago/report/?year=2019&month=11&date=23',
 '/nago/report/?year=2019&month=12&date=07']

In [55]:
home_url[1]


'/nago/report/?year=2019&month=03&date=30'

In [56]:
#リスト型にする
# pickup_links=[elem.attrs["href"] for elem in elems]
home_url=['https://www.football-lab.jp'+home_url[i] for i in range(len(home_url))]
home_url


['https://www.football-lab.jp/nago/report/?year=2019&month=03&date=02',
 'https://www.football-lab.jp/nago/report/?year=2019&month=03&date=30',
 'https://www.football-lab.jp/nago/report/?year=2019&month=04&date=20',
 'https://www.football-lab.jp/nago/report/?year=2019&month=04&date=28',
 'https://www.football-lab.jp/nago/report/?year=2019&month=05&date=12',
 'https://www.football-lab.jp/nago/report/?year=2019&month=05&date=26',
 'https://www.football-lab.jp/nago/report/?year=2019&month=06&date=22',
 'https://www.football-lab.jp/nago/report/?year=2019&month=07&date=07',
 'https://www.football-lab.jp/nago/report/?year=2019&month=07&date=20',
 'https://www.football-lab.jp/nago/report/?year=2019&month=08&date=10',
 'https://www.football-lab.jp/nago/report/?year=2019&month=08&date=24',
 'https://www.football-lab.jp/nago/report/?year=2019&month=08&date=30',
 'https://www.football-lab.jp/nago/report/?year=2019&month=10&date=05',
 'https://www.football-lab.jp/nago/report/?year=2019&month=10&da

In [57]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time


In [58]:
# ブラウザ起動
PATH = "C:\Program Files (x86)\chromedriver.exe"
# options = Options()
#options.add_argument("--incognito") # プライベートブラウズモードで開く

driver = webdriver.Chrome(executable_path = PATH)

# windows size設定(80%表示で最適な window size)
driver.set_window_size(840,1000)
#Webドライバーのタイムアウト時間を10秒に設定
wait = WebDriverWait(driver, 10)

In [59]:
def organizw_df(dfs):
    organize_df=[]
    for i in range(len(dfs)):
        # stats_df = pd.DataFrame(dfs[i][0][0])

        #データフレームを整理する
        stats_dfs[i].dropna(axis = 0, how = 'any', inplace=True) #一つでもNanが行に含まれてたら削除
        stats_dfs[i].set_index("スタッツ", inplace=True, drop=True) #Dropでsetindexの列を残さない

        #置換
        stats_dfs[i].iloc[:,2] = stats_dfs[i].iloc[:,2].str.replace("m", "")
        stats_dfs[i].iloc[:,2] = stats_dfs[i].iloc[:,2].str.replace("%", "")
        stats_dfs[i].iloc[:,2] = stats_dfs[i].iloc[:,2].str.replace(",", "")
        stats_dfs[i].iloc[:,3] = stats_dfs[i].iloc[:,3].str.replace("m", "")
        stats_dfs[i].iloc[:,3] = stats_dfs[i].iloc[:,3].str.replace("%", "")
        stats_dfs[i].iloc[:,3] = stats_dfs[i].iloc[:,3].str.replace(",", "")
        stats_dfs[i].iloc[:,1] = stats_dfs[i].iloc[:,1].str.replace("%", "")
        stats_dfs[i].iloc[:,1] = stats_dfs[i].iloc[:,1].str.replace(")", "")
        stats_dfs[i].iloc[:,1] = stats_dfs[i].iloc[:,1].str.replace("(", "")
        stats_dfs[i].iloc[:,1] = stats_dfs[i].iloc[:,1].str.replace("-", "0")
        stats_dfs[i].iloc[:,4] = stats_dfs[i].iloc[:,4].str.replace("%", "")
        stats_dfs[i].iloc[:,4] = stats_dfs[i].iloc[:,4].str.replace(")", "")
        stats_dfs[i].iloc[:,4] = stats_dfs[i].iloc[:,4].str.replace("(", "")
        stats_dfs[i].iloc[:,4] = stats_dfs[i].iloc[:,4].str.replace("-", "0")

        #float型にする
        stats_dfs[i].iloc[:,0] = stats_dfs[i].iloc[:,0].astype(float)
        stats_dfs[i].iloc[:,1] = stats_dfs[i].iloc[:,1].astype(float)
        stats_dfs[i].iloc[:,2] = stats_dfs[i].iloc[:,2].astype(float)
        stats_dfs[i].iloc[:,3] = stats_dfs[i].iloc[:,3].astype(float)
        stats_dfs[i].iloc[:,4] = stats_dfs[i].iloc[:,4].astype(float)
        stats_dfs[i].iloc[:,5] = stats_dfs[i].iloc[:,5].astype(float)

        organize_df.append(stats_dfs[i])

    organize_df[1]

In [60]:
def get_table(urls):
    dfs=[]
    for url in urls[1:4]: #一旦３つくらい
        driver.get(url)
    # CLASS名指定したページ上の要素が読み込まれるまで待機（15秒でタイムアウト判定）
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CLASS_NAME, 'statsTbl6 ')))
        elem_table = driver.find_element_by_xpath("/html/body/article/div[10]/table[2]")
        time.sleep(3)
        html = elem_table.get_attribute('outerHTML')
        df = pd.read_html(html) #ここではリスト

        #対戦チームの
        elem_home = driver.find_element_by_css_selector('#nago > article > div:nth-child(10) > div:nth-child(6) > em:nth-child(1)')
        elem_home_team = driver.find_element_by_css_selector('#nago > article > div:nth-child(10) > div:nth-child(6) > em:nth-child(2) > span.dsktp')
        elem_away = driver.find_element_by_css_selector('#nago > article > div:nth-child(10) > div:nth-child(6) > em:nth-child(4)')
        elem_away_team = driver.find_element_by_css_selector('#nago > article > div:nth-child(10) > div:nth-child(6) > em:nth-child(5) > span.dsktp')

        dfs.append([df,elem_home.text,elem_home_team.text,elem_away.text,elem_away_team.text])
        return dfs
    # teams_home_away.append([elem_home.text,elem_home_team.text,elem_away.text,elem_away_team.text])

In [78]:
dfs = get_table(home_url)
dfs

[[[          今季平均         成功率          総数  Unnamed: 3      総数.1    成功率.1  \
   0       ゴール期待値      ゴール期待値      ゴール期待値      ゴール期待値       NaN      NaN   
   1        1.583           -        2.24      ゴール期待値     1.447        -   
   2         シュート        シュート        シュート        シュート       NaN      NaN   
   3         14.2     (14.3%)          21        シュート        13   (0.0%)   
   4       枠内シュート      枠内シュート      枠内シュート      枠内シュート       NaN      NaN   
   5          4.5           -           7      枠内シュート         3        -   
   6    PKによるシュート   PKによるシュート   PKによるシュート   PKによるシュート       NaN      NaN   
   7          0.0           -           0   PKによるシュート         0        -   
   8           パス          パス          パス          パス       NaN      NaN   
   9        609.0     (84.2%)         652          パス       428  (73.4%)   
   10         クロス         クロス         クロス         クロス       NaN      NaN   
   11        14.8     (28.6%)          14         クロス        17  (35.3%)   
   12       

In [81]:
dfs[0][]

[[          今季平均         成功率          総数  Unnamed: 3      総数.1    成功率.1  \
  0       ゴール期待値      ゴール期待値      ゴール期待値      ゴール期待値       NaN      NaN   
  1        1.583           -        2.24      ゴール期待値     1.447        -   
  2         シュート        シュート        シュート        シュート       NaN      NaN   
  3         14.2     (14.3%)          21        シュート        13   (0.0%)   
  4       枠内シュート      枠内シュート      枠内シュート      枠内シュート       NaN      NaN   
  5          4.5           -           7      枠内シュート         3        -   
  6    PKによるシュート   PKによるシュート   PKによるシュート   PKによるシュート       NaN      NaN   
  7          0.0           -           0   PKによるシュート         0        -   
  8           パス          パス          パス          パス       NaN      NaN   
  9        609.0     (84.2%)         652          パス       428  (73.4%)   
  10         クロス         クロス         クロス         クロス       NaN      NaN   
  11        14.8     (28.6%)          14         クロス        17  (35.3%)   
  12        直接ＦＫ        直

In [ ]:
def change_column_name(dfs):
    #列名をそれぞれ変える
    change_dfs=[]
    for i in range(len(dfs)):
        stats_df = pd.DataFrame(dfs[i][0][0])
            # カラム名変更
        columns_name = [dfs[i][1]+"_"+dfs[i][2]+"_2019平均",
                            dfs[i][1]+"_"+dfs[i][2]+"_成功率",
                            dfs[i][1]+"_"+dfs[i][2]+"_総数",
                            "スタッツ",
                            dfs[i][3]+"_"+dfs[i][4]+"_総数",
                            dfs[i][3]+"_"+dfs[i][4]+"_成功率",
                            dfs[i][3]+"_"+dfs[i][4]+"_2019平均"]
        stats_df.columns = columns_name
        change_dfs.append(stats_df)
        # stats_df.dropna(axis = 0, how = 'any', inplace=True) #一つでもNanが行に含まれてたら削除
        # stats_df.set_index("スタッツ", inplace=True, drop=True) #Dropでsetindexの列を残さない
        return change_dfs

In [ ]:
dfs1 = change_column_name(dfs)

In [ ]:
dfs[0][0][0]

,HOME_名古屋グランパス_2019平均,HOME_名古屋グランパス_成功率,HOME_名古屋グランパス_総数,スタッツ,AWAY_北海道コンサドーレ札幌_総数,AWAY_北海道コンサドーレ札幌_成功率,AWAY_北海道コンサドーレ札幌_2019平均
0,ゴール期待値,ゴール期待値,ゴール期待値,ゴール期待値,NaN,NaN,NaN
1,1.583,-,2.24,ゴール期待値,1.447,-,1.742
2,シュート,シュート,シュート,シュート,NaN,NaN,NaN
3,14.2,(14.3%),21,シュート,13,(0.0%),14.000
4,枠内シュート,枠内シュート,枠内シュート,枠内シュート,NaN,NaN,NaN
5,4.5,-,7,枠内シュート,3,-,4.900
6,PKによるシュート,PKによるシュート,PKによるシュート,PKによるシュート,NaN,NaN,NaN
7,0.0,-,0,PKによるシュート,0,-,0.100
8,パス,パス,パス,パス,NaN,NaN,NaN
9,609.0,(84.2%),652,パス,428,(73.4%),500.400


In [ ]:
organizw_df(dfs)

NameError: name 'stats_dfs' is not defined